<a href="https://colab.research.google.com/github/Madhukar-Palakaveeti/deep_learning/blob/main/nninitial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
a = np.array([1,2,3,4])
b = np.array([[1,2,3,4],[5,6,7,8]])
c = np.array([[[1,2,3,4],[5,6,7,8]],[[2,3,4,5],[4,5,6,7]]])
print(a.shape)
print(b.shape)
print(c.shape)

(4,)
(2, 4)
(2, 2, 4)


In [20]:
z = np.zeros([3,2,4])
print(z)
print(z.shape)

[[[0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]]]
(3, 2, 4)


In [21]:
print(np.dot(b,a))
print(np.dot(c,a))

[30 70]
[[30 70]
 [40 60]]


In [29]:
I = np.identity(4)
print(I)
print(c)
print(I.shape)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[[1 2 3 4]
  [5 6 7 8]]

 [[2 3 4 5]
  [4 5 6 7]]]
(4, 4)


In [28]:
print(np.dot(c,I))

[[[1. 2. 3. 4.]
  [5. 6. 7. 8.]]

 [[2. 3. 4. 5.]
  [4. 5. 6. 7.]]]


In [30]:
inputs = [1,2,3,2.5]
weights = [[0.2,0.8,-0.5,1.0],[0.5,-0.91,0.26,-0.5],[-0.26,-0.27,0.17,0.87]]
biases = [2,3,0.5]
print(np.dot(weights,inputs)+biases)

[4.8   1.21  2.385]


In [31]:
X = np.array([[1,2,3,2.5],[2.0,5.0,-1.0,2.0],[-1.5,2.7,3.3,-0.8]])
print(X)

[[ 1.   2.   3.   2.5]
 [ 2.   5.  -1.   2. ]
 [-1.5  2.7  3.3 -0.8]]


In [32]:
np.random.seed(0)


In [33]:
class Layer_Dense:
  def __init__(self,n_inputs,n_neurons):
    self.weights = 0.10 * np.random.randn(n_inputs,n_neurons)
    self.biases = np.zeros((1,n_neurons))

  def forward(self,input):
    self.output = np.dot(input, self.weights)+ self.biases

layer1 = Layer_Dense(4,5)
layer2 = Layer_Dense(5,2)
layer1.forward(X)
print(layer1.output)


[[ 0.10758131  1.03983522  0.24462411  0.31821498  0.18851053]
 [-0.08349796  0.70846411  0.00293357  0.44701525  0.36360538]
 [-0.50763245  0.55688422  0.07987797 -0.34889573  0.04553042]]


In [34]:
layer2.forward(layer1.output)
print(layer2.output)

[[ 0.148296   -0.08397602]
 [ 0.14100315 -0.01340469]
 [ 0.20124979 -0.07290616]]
